In [4]:
from bs4 import BeautifulSoup
import mechanize
import urllib2 
import cookielib
import requests
import csv
import pandas as pd
from urllib2 import Request, urlopen
from PyPDF2 import PdfFileWriter, PdfFileReader
from StringIO import StringIO
import ssl
from subprocess import call
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFParser
from pdfminer.pdfpage import PDFDocument
from pdfminer.pdfdevice import PDFDevice
import os
from subprocess import call


class QueryClass(object):
    def __init__(self, dictid, query_id, equipment_type, url, h3title, score, h3_score, url_score, maintenanceP_score, modelP_score, mgfP_score, manualP_score, content_score, url_mgf, result_num,  is_readable, winner, errorMsg, is_service_manual, loser, maintenanceP, modelP, manualP, mgfP):
        self.dictid = dictid
        self.query_id = query_id
        self.equipment_type = equipment_type
        self.url = url
        self.h3title = h3title
        self.errorMsg = errorMsg
        self.score = score
        self.h3_score = h3_score
        self.url_score = url_score
        self.maintenanceP_score = maintenanceP_score
        self.modelP_score = modelP_score
        self.manualP_score = manualP_score
        self.mgfP_score = mgfP_score
        #self.equipmentP_score = equipmentP_score
        self.is_readable = is_readable
        self.content_score = content_score
        self.result_num = result_num
        self.is_service_manual = is_service_manual
        self.url_mgf = url_mgf
        self.loser = loser
        self.winner = winner
        self.maintenanceP = maintenanceP
        self.modelP = modelP
        self.manualP = manualP
        self.mgfP = mgfP
                
#Returns list of elements to query, read from file specified by filename
def readList(filename):
    query_list = []            
    with open(filename, 'rb') as csvfile:
        filereader = csv.reader(csvfile, quotechar='|')
        for row in filereader:
            query_list.append(row)
        return query_list
    
def readable():
    path = "/Users/marianahu/Documents/decrypted_manual.pdf"
    try:
        pdf = PdfFileReader(open(path, "rb"))
        fp = file(path, 'rb')
    except Exception, e:   
        print "Exception in readable function happened %s" %e
        readable = 10 #exception
        return readable
    else:   
    #num_of_pages = pdf.getNumPages()
        readable = 0
  
    for i in range(20):
        inside = [i]
        pagenos=set(inside)
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ""
        maxpage = 0
        caching = True
        try:
            for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpage,password = password, caching=caching, check_extractable=True):
                interpreter.process_page(page)
                text = retstr.getvalue()
                text = text.decode("ascii","replace")
        except Exception, e:   
            print "Exception in readable function happened %s" %e
            readable = 10 #exception
            return readable
        else:    
            if "the" in text.lower():
                readable = 1
                break
                return readable
            else:
                continue
    return readable

def service_manual():
    path = "/Users/marianahu/Documents/decrypted_manual.pdf"
    try:
        decrypted_manual_exists = os.path.exists(path)
        pdf = PdfFileReader(open(path, "rb"))
        fp = file(path, 'rb')
    except Exception, e:   
        print "Exception in service manual function happened %s" %e
        service_manual = -1
        return service_manual
    else:    
        num_of_pages = 3
        service_manual = 0
        for i in range(num_of_pages):
            inside = [i]
            pagenos=set(inside)
            rsrcmgr = PDFResourceManager()
            retstr = StringIO()
            codec = 'utf-8'
            laparams = LAParams()
            device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            password = ""
            maxpage = 0
            caching = True
            try:
                for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpage,password = password, caching=caching, check_extractable=True):
                    interpreter.process_page(page)
                    text = retstr.getvalue()
                    text = text.decode("ascii","replace")
            except Exception, e:
                print "Exception in service_manual function happened %s" %e
                #exception
                service_manual = -1
                return service_manual
            else:
                if "technical service manual" in text.lower() or "technical manual" in text.lower():
                    #technical service manual
                    service_manual = 2 
                    print "It's a technical service manual"
                    return service_manual
                elif "service manual" in text.lower():
                    #service manual
                    service_manual = 1 
                    print "It's a service manual"
                    return service_manual
                else:
                    continue
        print "It's NOT a service manual"
        return service_manual

def download_pdf(url):
    downloaded_manual_exists = False
    try:
        response = requests.get(url)
    except Exception, e:
        print "Exception in download_pdf function happened %s" %e
        return downloaded_manual_exists
    else:
        try:
            with open('/Users/marianahu/Documents/downloaded_manual.pdf', 'wb') as f:
                f.write(response.content)
        except Exception, e:
            print "Exception in download_pdf function (writing the pdf) happened %s" %e
            return downloaded_manual_exists
        else:
            path = "/Users/marianahu/Documents/downloaded_manual.pdf"
            downloaded_manual_exists = os.path.exists(path)
            print "downloaded manual exists? " +str(downloaded_manual_exists)
            return downloaded_manual_exists
    
def decrypt_pdf():
    decrypted_manual_exists = False
    try:
        call('qpdf --password=%s --decrypt %s %s' %('', 'downloaded_manual.pdf', 'decrypted_manual.pdf'), shell=True)
    except Exception, e:
        print "Exception in decrypt_pdf function happened %s" %e
        return downloaded_manual_exists
    else:
        path = "/Users/marianahu/Documents/decrypted_manual.pdf"
        decrypted_manual_exists = os.path.exists(path)
        print "decrypted manual exists? " +str(decrypted_manual_exists)
        return decrypted_manual_exists
    
def empty_list(word):
    empty_list = []
    word_len = len(word)
    for l in range(word_len): 
        empty_list.append(l)
        empty_list[l]=0
    return empty_list


def getNewMgfName(mgf_name):
    mgf_name = mgf_name.lower()
    empty = ['unknown', 'pending', 'none']
    if any(empty_words in mgf_name for empty_words in empty):
        return "skip"
        
    middle = [ ", ", " sub ", "(", " div ", " prod ", " inc ", " incorporated",  " a ", " an ", " aka ", " corp of america", "/", " /", "s/a", "t/a"]
    end = [" co", " co "," co.", "div", " div.",  "corp", "corp ", "corporation", "corporation ", "llc", "lc", " inc", " inc.", "inc ", "ltd", "pty ltd", "s.p.a"]

    for words in middle:
        if words in mgf_name:
            mgf_name = cutMgfName(mgf_name, words)
    for palabras in end:
        if mgf_name.endswith(palabras):
            mgf_name = cutMgfName(mgf_name, palabras)
            
    if "&" in mgf_name:
        print "& in mgf name"
        mgf_name = mgf_name.replace("&", "and")
    if " scient " in mgf_name:
        mgf_name = mgf_name.replace("scient", "scientific")  
        
    unwanted = [' and', ' america ', ' medical products', ' medical systems', ' u.s.a.', ' usa ', '. ' ]
    for word in unwanted:
        if word in mgf_name:
            mgf_name = mgf_name.replace(word, "")
    print "new mgf name is "+mgf_name
    return mgf_name
        
def cutMgfName(mgf_name, word):
    try:
        index = mgf_name.index(word)
        mgf_name = mgf_name[0:index]
    except Exception, e:
        print "Exception  in CutMgfName function happened %s" %e
        return mgf_name
    else:
        return mgf_name

def mgfWords(mgf_name):
    mgf_words = mgf_name.split()
    return mgf_words

def isModelUnknown(model_name):
    model_name = model_name.lower()
    empty = ['unknown', 'pending', 'none']
    if any(empty_words in model_name for empty_words in empty):
        return "skip"
    else:
        return model_name
    
    
def getModels(model_name):
    model_names = []
    model_names.append(model_name)

    if "-" in model_name:
        model2 = model_name.replace("-", " ")
        model_names.append(model2)
        model3 = model_name.replace("-", "")
        model_names.append(model3)
        model_words = model2.split()
        for word in model_words:
            model_names.append(word + " ")
    else:
        model_words = model_name.split()
        if len(model_words) > 1:
            for word in model_words:
                model_names.append(word)
    #print model_names
    return model_names

def bestResult(results):
    #results is a list of obj QueryClass
    print "Calculating best results. These are the results received and their content scores:"
    for element in results:
        print element.content_score
   
    if results == "skip" or len(results)==0:
        best = []
        return best
    
    best = []
    #if there is only one result in the list, and it is not a loser, then make it the best result
    if len(results) == 1:
        if results[0].loser == 0:
            best = [results[0]]
            print "Length of results is 1 and best is of type "+str(type(best))
            return best
        else:
            print "The only result was a loser"
            best = []
            return best
        
    else: #if there is more than one result in the list, check whether there is any winner
        maxval = None
        indices = []
        winner_result = 0
        #First check if there is any winner among results. If yes, set winner_result = 1
        for index, result in enumerate(results):
            if result.winner == 1:
                indices = [index]
                best_index = indices[0]
                best = [results[best_index]]
                winner_result = 1
                print "We found a winner result of index "+ str(best_index)
                break
        #If no winner results were found among the results... compare content scores and determine best result
        if winner_result == 0:
            print "We didn't find a winner result so we'll check for the highest content scores"
            for index, result in enumerate(results):
                if result.loser == 0:
                    if maxval is None or result.content_score > maxval:
                        indices = [index]
                        maxval = result.content_score
                    elif result.content_score == maxval: #if the next result has same score as the other one, append its index to the list
                        indices.append(index)
                else:
                    continue
            
            if len(indices)==0: #if no winner results and all others are losers, return empty stuff
                print "The results were all losers"
                best = []
                return best
            
            else:
                print "We found the following indices for the highest scores:"
                for index in indices:
                    print str(index)
        
                print "The length of the indices list is "+str(len(indices))
                if len(indices)>1:
                    best_index = min(indices)
                else:
                    best_index = indices[0]

        print "The best index is "+str(best_index)

        best = [results[best_index]]

        print "The best result is number "+ str(best[0].result_num) + " and its content score is " +str(best[0].content_score)

        if best_index > 0: #if the best index is not [0], check whether there is a non readable result with smaller index than best_index
            print "checking for any non-readable results..."
            for i in range(best_index):
                print "i is: "+str(i)
                if results[i].is_readable == 0:
                    print "We found a non-readable result of smaller index than the best_index"
                    nonreadable_result = results[i]
                    best.append(nonreadable_result)
                    print "Result number "+ str(nonreadable_result.result_num) +" was added as a non-readable result"

                else:
                    print "We did not find any non-readable result of smaller index than the best_index"

            print "The result_num that will be returned are: "
            for each in best:
                print str(each.result_num)
            #print "Type of best is: "+str(type(best))
            return best

        else:
            print "The only best result is no.: "+ str(best[0].result_num)
            print "The best result has content score: "+ str(best[0].content_score)
            return best   

def delete_pdf():
    try:
        os.remove("/Users/marianahu/Documents/decrypted_manual.pdf")
        os.remove("/Users/marianahu/Documents/downloaded_manual.pdf")
    except:
        return

    
def fake_stuff(url): #checks for fake pdfs, those that are blank but contains a filename + .pdf
    fake = 0
    writer = PdfFileWriter()
    try:
        req = urllib2.Request(url , headers={'User-Agent': 'Mozilla/5.0'})
        if "https" not in url.lower():
            print "no https in url"
            remoteFile = urlopen(req).read()
        else:
            print "https in url"
            ctx = ssl.create_default_context()
            ctx.check_hostname = False
            ctx.verify_mode = ssl.CERT_NONE
            remoteFile = urlopen(req, context=ctx).read()
        
        memoryFile = StringIO(remoteFile)
        pdfFile = PdfFileReader(memoryFile)
        if pdfFile.isEncrypted:
            pdfFile.decrypt('')
        num_of_pages = pdfFile.getNumPages()
        
    except Exception, e:
        print "Exception in fake_stuff function happened %s" %e 
        return fake
    else:
        for p in range(num_of_pages):
            page = pdfFile.getPage(p)
            try:
                text = page.extractText()
            except Exception, e:
                #print 'Exception #3 happened %s' %e
                continue
            else:
                if " .pdf" in text.lower() or "ebook" in text.lower():
                    fake = 1
                    return fake
                else:
                    continue
        return fake
    
def find_index(url):
    chars = list(url)
    i=0
    idx = 0
    for char in chars:
        idx = idx + 1
        if char == ".":
            i=i+1
        if i==2:
            return idx
        else:
            continue
        
def get_Scores(model, mgf):
    mgf_len = len(mgf)
    maintenanceP_score = 0
    manualP_score = 0
    mgfP_score = 0
    modelP_score = 0
    #eqP_score = 0
    maintenanceP = 0
    manualP = 0
    mgfP = 0
    modelP = 0
    #eqP = 0print "Entering get_Scores with " + str(model) + ", " + str(mgf) + ", " #+ str(eq)
    try:
        path = "/Users/marianahu/Documents/decrypted_manual.pdf"
        pdf = PdfFileReader(open(path, "rb"))
        fp = file(path, 'rb')
        num_of_pages = pdf.getNumPages()
    except Exception, e:   
        print "Exception in get_Scores method happened %s" %e
        return maintenanceP_score, manualP_score, modelP_score, mgfP_score, maintenanceP, modelP, manualP, mgfP

    else:
        model_len = len(model)

        #creating empty lists for model and mgf. Later they will contain variations of model and mgf names
        modelP = empty_list(model)
        mgfP = empty_list(mgf)

        for i in range(num_of_pages):
            if num_of_pages < 150:
                #print "This is page: " +str(i)+ " of " +str(num_of_pages)
                inside = [i]
                pagenos = set(inside)
                rsrcmgr = PDFResourceManager()
                retstr = StringIO()
                codec = 'utf-8'
                laparams = LAParams()
                device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
                interpreter = PDFPageInterpreter(rsrcmgr, device)
                password = ""
                maxpage = 0
                caching = True
                try:
                    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpage,password = password, caching=caching, check_extractable=True):
                        interpreter.process_page(page)
                        text = retstr.getvalue()
                        text = text.decode("ascii","replace")
                except Exception, e:   
                    print "Exception in get_Scores function happened %s" %e
                    return maintenanceP_score, manualP_score, modelP_score, mgfP_score
                else:
                    #print text.lower()
                    if "maintenance" in text.lower() or "calibra" in text.lower():
                        maintenanceP = maintenanceP + 1
                    if "manual" in text.lower():
                        manualP = manualP + 1
                    #if any(eqword in text.lower() for eqword in eq):
                    #    eqP = eqP + 1

                    #In each pdf page, check presence of all model and mgf variations (range of "l"). 
                    #Give 1 point for every time one of the variations appear in the text

                    for l in range(model_len):
                        if model[l] in text.lower():
                            modelP[l] = modelP[l] + 1

                    for l in range(mgf_len):
                        if mgf[l] in text.lower():
                            mgfP[l] = mgfP[l] + 1

        print "Done counting words in each page. Now let's summarize the ____P: "
        print "maintenanceP :" + str(maintenanceP)
        print "manualP :" + str(manualP)
        #print "eqP :"+str(eqP)
        print "mgfP: "
        for idx, val in enumerate(mgfP):
            print (idx, val)
        print "modelP: "
        for idx, val in enumerate(modelP):
            print (idx, val)

        #Calculate the ___P_scores
        if model_len == 1:
            if modelP[0] > 0:
                modelP_score = 2
            else:
                modelP_score = 0

        #new_modelP is a subset of the modelP (list of scores). The subset takes modelP from index 1 until the end
        new_modelP = modelP[1:model_len]
        print "These are the scores of new_modelP"   
        for score in new_modelP:
            print str(score)

        #if model_len is between 1 and 4, modelP_score = 2 if modelP[0] >0... 
        #otherwise, modelP_score = 1 if any modelP[1] through bmodelP[4] is >0
        if model_len > 1 and model_len <5:
            if modelP[0] > 0:
                modelP_score = 2
            else:
                for variation_score in new_modelP:
                    print "Variation score: " +str(variation_score)
                    if variation_score > 0:
                        modelP_score = modelP_score + 1
            print "modelP_score: "+ str(modelP_score)

            #elif any (p > 0 for p in new_modelP):
                #elif modelP[1] > 0 or modelP[2] > 0 or modelP[3] > 0:
                #       modelP_score = 1
                #  else:
                #     modelP_score = 0

        #new_modelP2 is a subset of the modelP. The subset takes modelP from index 3 until the end
        new_modelP2 = modelP[3:model_len]
        #if model_len>5, modelP_score = 2 if any modelP[0] through modelP[2] >0... 
        #otherwise, modelP_score = 1 for modelP[3] through the last modelP[index]
        if model_len >= 5:
            if modelP[0] > 0 or modelP[1] > 0 or modelP[2] > 0:
                modelP_score = 2
            #elif modelP[3] > 0 or modelP[4] > 0 or modelP[5] > 0 or modelP[6] > 0:
            else:
                for variation_score2 in new_modelP2:
                    print "Variation score2: " +str(variation_score2)
                    if variation_score2 > 0:
                        modelP_score = modelP_score + 1
            print "modelP_score: "+ str(modelP_score)

            #elif any (p > 0 for p in new_modelP2):
                #modelP_score = 1
            #else:
                #modelP_score = 0

        if mgf_len == 1:
            if mgfP[0] > 0:
                mgfP_score = 2
            else:
                mgfP_score = 0

        if mgf_len > 1:
            if mgfP[0] > 0:
                mgfP_score = 2
            elif any (p > 0 for p in mgfP):
                mgfP_score = 1
            else:
                mgfP_score = 0

        if maintenanceP > 2:
            maintenanceP_score = 1
        else: 
            maintenanceP_score = 0

        if manualP > 0:
            manualP_score = 1
        else: 
            manualP_score = 0

        #if eqP > 0:
        #    eqP_score = 1
        #else:
        #    eqP_score = 0

        print "Here are the ___P_scores:"
        print "MaintenanceP_score" +str(maintenanceP_score)
        print "ManualP_score " +str(manualP_score)
        print "ModelP_score " +str(modelP_score)
        print "MgfP_score " +str(mgfP_score)
        #print "EqP_score " +str(eqP_score)

        return maintenanceP_score, manualP_score, modelP_score, mgfP_score, maintenanceP, modelP, manualP, mgfP

def getResults(element, number_results):
    dictid = element[3]
    results = []
    mgf_name = getNewMgfName(element[0])
    if mgf_name == "skip":
        empty_obj = QueryClass(dictid, "Unknown mgf name"," ", " ", " ", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, "Unknown/Pending/None", 0, 1, 0, 0, 0, 0)
        results.append(empty_obj)
        return results
            
    mod = isModelUnknown(element[1])
    if mod == "skip":
        empty_obj = QueryClass(dictid, "Unknown model name"," ", " ", " ", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, "Unknown/Pending/None", 0, 1, 0, 0, 0, 0)
        results.append(empty_obj)
        return results
    
    model_id = element[1].replace(' ', '+')
    print "This is the model_ID: " +model_id
    mgf_id = mgf_name.replace(' ', '+')
    print "This is the mgf_ID: " + mgf_id
    query_id = element[0] + " "+element[1]
    
    print "Requesting url: https://www.google.com/search?q="+mgf_id+"+"+model_id+"+service+manual+filetype:pdf"
    try:
        r  = requests.get("https://www.google.com/search?q="+mgf_id+"+"+model_id+"+service+manual+filetype:pdf")
        data = r.content
        soup = BeautifulSoup(data)
    except Exception, e:
        print "Exception in requesting Search url happened: %s" %e
        errorMsg = e
        obj0 = QueryClass(dictid, query_id," ", " ", " ",  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,  errorMsg, 0, 1, 0, 0, 0, 0)
        results.append(obj0)
        return results
    else:
        h3tags = soup.find_all('h3')
        print "number of h3tags: "+ str(len(h3tags))
        if h3tags:
            i=0
            for i in range(number_results):
                if i< len(h3tags):
                    result_num = i+1
                    winner = 0
                    loser =  0
                    is_service_manual = 0
                    print " "
                    print "This is i: " +str(result_num)
                    h3tag = h3tags[i]
                    h3title = h3tag.text.encode('utf-8')
                    print "This is the h3title: " +h3title
                    print " "
                    
                    if "brochure" in h3title.lower() or "catalog" in h3title.lower():
                        obj0 = QueryClass(dictid, query_id," ", " ", " ",  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, "Catalog or brochure", 0, 1, 0, 0, 0, 0)
                        results.append(obj0)
                        continue
                            
                    atag = h3tag.a
                    url= atag.get('href')
                    url = urllib2.unquote(url)
                    #url = url.lower()
                    try:
                        start = url.index('q=')
                        end = url.index('&', start)        
                    except Exception, e:
                        print "Exception in stripping url happened %s" %e
                        errorMsg = e
                        obj0 = QueryClass(dictid, query_id, " ", " ", " ", 0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 1, 0, errorMsg, 0, 1, 0, 0, 0, 0)
                        results.append(obj0)
                        continue
                    else:
                        #Obtaining the url
                        url= url[start+2:end]
                        print "This is the url: "+url
                        #Checking if url has fake domain
                        fake_url = ["muvenbeauconc","cphs.us", "hiscodoty", "wordpress", "kitschmoog", "daydream", "ryokanblog", "ilmsekho", "realholidayindia"]
                        if any(fake in url.lower() for fake in fake_url):
                            obj0 = QueryClass(dictid, query_id," ", " ", " ", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, "Fake link per url", 0, 1, 0, 0, 0, 0)
                            print "Fake url per content"
                            results.append(obj0)
                            continue    
                        is_fake = fake_stuff(url)
                        if is_fake == 1:
                            obj0 = QueryClass(dictid, query_id," ", " ", " ", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, "Fake link per content", 0, 1, 0, 0, 0, 0)
                            print "Fake url per content"
                            results.append(obj0)
                            continue

                        #obtaining components of model names string
                        model = element[1] + " "
                        model_names = []
                        model_names = getModels(model.lower())
                        model_len = len(model_names)
                        print "Model names: "
                        print model_names
                        
                        #Obtaining components of equipment type string
                        equipment_type = element[2].lower()
                        #equipment_words = equipment_type.split()
                        #print "Equipment words: "
                        #print equipment_words
    
                        #Obtaining mgf words (stripping off certain words, and separating each word into a list)
                        mgf_words = mgfWords(mgf_name)
                        print "Mgf words: "
                        print mgf_words
                        
                        #Checking if url domain contains mgf names
                        index = find_index(url)
                        domain = url[4:index]
                        print "Domain: " +domain
                        
                        if any (mgfword in domain for mgfword in mgf_words):
                            url_mgf = 1
                        else:
                            url_mgf = 0
                    
                        #Analyzing PDF content
                        downloaded_file_exists = download_pdf(url)
                        if downloaded_file_exists == True:
                            decrypted_file_exists = decrypt_pdf()
                            if decrypted_file_exists == True:
                                is_readable = readable()
                                print "Readable? " + str(is_readable)

                            elif decrypted_file_exists == False:
                                print "Could not decrypt PDF"
                                delete_pdf()
                                obj0 = QueryClass(dictid, query_id, " ", " ", " ", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, "Could not decrypt pdf", 0, 1, 0, 0, 0, 0)
                                results.append(obj0)
                                continue
                        elif downloaded_file_exists == False:
                            print "Could not download PDF"
                            obj0 = QueryClass(dictid, query_id, " ", " ", " ", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, "Could not download pdf", 0, 1, 0, 0, 0, 0)
                            results.append(obj0)
                            continue
                        
                        #We have downloaded and decrypted the pdf, and we know whether it is readable or not. 
                        #If it is readable, we will obtain their content scores and check whether there are any winners or losers
                        if is_readable != 0: #i.e. pdf is readable             
                            scores = get_Scores(model_names, mgf_words)
                            maintenanceP_score = scores[0] 
                            manualP_score = scores[1] 
                            modelP_score = scores[2] 
                            mgfP_score = scores[3]
                            #equipmentP_score = scores[4]
                            maintenanceP = scores[4]
                            modelP = scores[5]
                            manualP = scores[6]
                            mgfP = scores[7]
                            is_service_manual = service_manual()
                            print "is_Service_manual: "+str(is_service_manual)
                            
                            #Is it a winner?
                            if modelP_score == 2 and maintenanceP_score == 1 and mgfP_score > 0 and is_service_manual > 0:
                                winner = 1
                                print "WE FOUND A WINNER!"
                            elif modelP_score == 1 and maintenanceP_score == 1 and mgfP_score > 0 and is_service_manual > 0 and url_mgf == 1:
                                winner = 1
                                print "WE FOUND A WINNER! (url_mgf)"                        
                            else:
                                winner = 0
                            
                            #Is it a loser? It's a loser if either maintenance is not there, or maintenance is there
                            #but neither mgf or model are there
                            if readable == 1 and maintenanceP_score == 0:
                                loser = 1
                                print "We found a loser!"
 
                        if is_readable == 0:
                            maintenanceP_score =0
                            modelP_score = 0
                            manualP_score = 0
                            mgfP_score = 0
                            #equipmentP_score = 0
                            content_score = 0
                            is_service_manual = 0
                            maintenanceP=0
                            modelP=0
                            manualP=0
                            mgfP=0
                            errorMsg = "PDF not readable"
                            print "PDF not readable"
                        
                        content_score = maintenanceP_score + modelP_score+manualP_score+ mgfP_score #+ equipmentP_score 
                        print "The content score is: "+ str(content_score)
                        
                        if is_readable == 1 and content_score < 6:
                            loser = 1
                            print "We found a loser!"
                            
                        #Analyzing URL 
                        url_score = 0
                        if len(mgf_words) == 1:
                            if mgf_words[0] in url.lower():
                                print "this mgf word was found in the url: "+mgf_words[0]
                                url_score = url_score + 1
                        else:
                            if any(words in url.lower() for words in mgf_words):
                                url_score = url_score + 1
                        
                        good_url = ["frankshospital","medwrench", "santoslab", "medonecapital.com/manuals/techmanuals"]
                        if any(good in url.lower() for good in good_url):
                            url_score = url_score + 1
                            
                        if url_mgf == 1:
                            url_Score = url_score + 1
                            
                        if model_len == 1:
                            if model_names[0] in url.lower():
                                url_score =  url_score + 2
                        elif model_len <5: #if the model name has more than 1 word and no dashes
                            for idx, val in enumerate(model_names):
                                if val in url.lower():
                                    print(idx, val)
                                    if idx == 0:
                                        url_score =  url_score + 2
                                    else:
                                        url_score =  url_score + 1
                                    break
                        elif model_len >= 5: #if the model name has at least 1 dash
                               for idx, val in enumerate(model_names):
                                if val in url.lower():
                                    print(idx, val)
                                    if idx < 3:
                                        url_score = url_score + 2
                                    else:
                                        url_score = url_score + 1
                                    break    
                        
                        if 'service' in url.lower():
                            url_score = url_score + 1
                            
                        if 'technical' in url.lower():
                            url_score = url_score + 1
                            
                        if 'manual' in url.lower():
                            url_score = url_score + 1
                        
                        print "The url score is: " +str(url_score)
                            
                        #Analyzing h3 title
                        h3_score = 0
        
                        if model_len == 1:
                            if model_names[0] in h3title.lower():
                                print "this model word was found in the h3: "+model_names[0]
                                h3_score =  h3_score + 2
                        elif model_len <5: #if the model name has more than 1 word and no dashes
                            for idx, val in enumerate(model_names):
                                if val in h3title.lower():
                                    print "Model name was found in h3 title"
                                    print(idx, val)
                                    if idx == 0:
                                        h3_score =  h3_score + 2
                                    else:
                                        h3_score =  h3_score + 1
                                    break
                        elif model_len >= 5: #if the model name has at least 1 dash
                               for idx, val in enumerate(model_names):
                                if val in h3title.lower():
                                    print "Model name was found in h3 title"
                                    print(idx, val)
                                    if idx < 3:
                                        h3_score =  h3_score + 2
                                    else:
                                        h3_score =  h3_score + 1
                                    break
                                
                        if len(mgf_words) == 1:
                            if mgf_words[0] in h3title.lower():
                                h3_score =  h3_score + 1
                        else:
                            if any(words in h3title.lower() for words in mgf_words):
                                h3_score =  h3_score + 1
                        
                        if 'service' in h3title.lower():
                            h3_score = h3_score + 1
                            
                        if 'technical' in h3title.lower():
                            h3_score = h3_score + 1

                        if 'manual' in h3title.lower():
                            h3_score = h3_score + 1
                    
                        print "The h3 score is: " + str(h3_score)

                        score = h3_score + url_score + content_score
                        print "The total score is: " + str(score)
                    
                        obj2 = QueryClass(dictid, query_id, equipment_type, url, h3title, score, h3_score, url_score, maintenanceP_score, modelP_score, mgfP_score, manualP_score, content_score,url_mgf, result_num, is_readable, winner, "No error", is_service_manual, loser, maintenanceP, modelP, manualP, mgfP)
                        results.append(obj2)
                        print "This is the result number: " + str(result_num)
                        print "Total Score: "+ str(obj2.score)
                        print "h3 score: "+ str(obj2.h3_score)
                        print "url score: " +str(obj2.url_score)
                        print "maintenance score : " + str(obj2.maintenanceP_score)
                        print "model score: " +str(obj2.modelP_score)
                        print "manual score: " +str(obj2.manualP_score)
                        print "mgf score: " +str(obj2.mgfP_score)
                        #print "equipment score: " +str(obj2.equipmentP_score)
                        print "content score: " +str(obj2.content_score)
            delete_pdf()

            return results
        
        else:
            print "H3tags not found"         
            obj0 = QueryClass(dictid, query_id, " ", " ", " ", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,  "No H3 tags", 0, 1, 0, 0, 0, 0)
            results.append(obj0)
            return results

    #Returns number of pages containing specified word in the specified pdf doc

ImportError: C extension: hashtable not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext --inplace' to build the C extensions first.

In [ ]:
# delete_pdf()
input_list = readList('dictid list.csv')

output_list = []
all_results = []
best_results = []

sample_size = 50

#I call getResults for each row of the input_list (i.e. for each dictid)
# getResults returns a list of results saved in output_obj
for element in range(9646,11610):
    print "\n"
    print "element no. " + str(element)
    print 'element is=' +str(input_list[element])
    output_obj = getResults(input_list[element], 3)
    
    #i convert each result in output_obj into a dict, and then make a list of dicts (all_results)
    for obj in output_obj:
        obj = obj.__dict__
        all_results.append(obj)
    #all_results is a list of dictionaries. Each dictionary contains a result returned by getResults
    
    #the bestResult method takes the list of results (output_obj) and returns a list of the "best" results  (i.g. best)
    best = []
    best = bestResult(output_obj)
    print "Exited the bestResult method and concluded that:"
    #print "best is of type: "+ str(best) +" and length: "+str(len(best))
    
    #best is a list of the best results... i convert it into a list of  dictionaries
    #best_results is the list of dictionaries
    if len(best) == 0:
        print "No best result"
    if len(best) == 1:
        print "The best result is of length 1"
        #if best[0].errorMsg != "No error":
        best = best[0].__dict__
        best_results.append(best)
    elif len(best) > 1:
        print "The best result is of length "+str(len(best))
        for each in best:
            each= each.__dict__    
            best_results.append(each)   

In [ ]:
# print str(type(best_results))
df = pd.DataFrame(best_results, columns=['dictid', 'winner', 'loser', 'errorMsg', 'query_id','content_score', 'is_readable','equipment_type', 'url', 'h3title', 'score','result_num','is_service_manual',
                                         'h3_score','url_mgf' ,'url_score', 'maintenanceP_score', 'modelP_score', 'mgfP_score',
                                        'manualP_score', 'maintenanceP', 'modelP', 'manualP', 'mgfP'] )

df.to_csv('output_dictid_run8winner_best.csv', mode = 'a', header = False)


In [41]:
#print type(output_obj)

    
a = pd.DataFrame(all_results, columns=['dictid', 'winner', 'loser', 'errorMsg', 'query_id', 'content_score', 'is_readable', 'equipment_type', 'url', 'h3title', 'score','result_num','is_service_manual',
                                         'h3_score','url_mgf' ,'url_score', 'maintenanceP_score', 'modelP_score', 'mgfP_score',
                                        'manualP_score', 'equipmentP_score', 'maintenanceP', 'modelP', 'manualP', 'mgfP'])  

a.to_csv('output_dictid_run8winner_all.csv', mode = 'a', header = False)

In [ ]:
length = len(input_list) -1
print length

import random
sample = random.sample(range(length), 10)

output_list = []

for element in sample:
    print "\n"
    print "element no. " + str(element)
    print 'element is=' +str(input_list[element])
    output_obj = getResults(input_list[element], 3)
    #output_obj receives a list of 3 QueryClass objects
    for obj in output_obj:
        obj = obj.__dict__ #each QueryClass object in output_obj is converted into a dictionary
        output_list.append(obj) #output_list is a list of dictionaries
    